In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from scipy import spatial
import scipy.constants
from scipy.constants import pi
import logging
import traceback
import time
logging.basicConfig(level=logging.INFO)

In [2]:
def within(circle,point):
    if np.sqrt(np.dot(circle[0:2]-point,circle[0:2]-point))<circle[2]:
        return True
    else:
        return False

In [3]:
def nearest_cell(circles,centroids):
    cells_xy = circles[0][:,0:2]
    aggrs_xy = np.array(centroids)
    d = spatial.distance_matrix(cells_xy,aggrs_xy)
    cells_idx = []
    for id in range(d.shape[1]):
        i = np.argmin(d[:,id])
        if within(circles[0][i,:],aggrs_xy[id,:]):
            cells_idx.append(np.array([i,id]))#cell id, aggregate id
    cells_idx = pd.DataFrame(np.array(cells_idx),columns=["circle_index","agg_index"])
    return cells_idx

In [4]:
def cell_ids(circles,cell_labels):
    global version
    circles[0][:,0] = np.round(circles[0][:,0],decimals=-2)
    cdf = pd.DataFrame(circles[0][:,0:3], columns=['x', 'y','D_pix'])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    if version=='V1':
        cdf['x'][0:6]=1200
        cdf['x'][6:12]=1400
        cdf['x'][12:18]=1600
        cdf['x'][18:24]=1800
        cdf['x'][24:30]=2000
        cdf['x'][30:36]=2200
        cdf = cdf.sort_values(by=['x','y'])#counts down by column
        cdf['y'][0:6]=np.array([500,700,900,1100,1300,1500])
        cdf['y'][6:12]=np.array([500,700,900,1100,1300,1500])
        cdf['y'][12:18]=np.array([500,700,900,1100,1300,1500])
        cdf['y'][18:24]=np.array([500,700,900,1100,1300,1500])
        cdf['y'][24:30]=np.array([500,700,900,1100,1300,1500])
        cdf['y'][30:36]=np.array([500,700,900,1100,1300,1500])
        cdf = cdf.sort_values(by=['x','y'])#counts down by column
    else:
        cdf['x'][0:4]=1200
        cdf['x'][4:8]=1500
        cdf['x'][8:12]=1800
        cdf['x'][12:16]=2100
        cdf['x'][16:20]=2400
        cdf = cdf.sort_values(by=['x','y'])#counts down by column
        cdf['y'][0:4]=np.array([700,1000,1300,1600])
        cdf['y'][4:8]=np.array([700,1000,1300,1600])
        cdf['y'][8:12]=np.array([700,1000,1300,1600])
        cdf['y'][12:16]=np.array([700,1000,1300,1600])
        cdf['y'][16:20]=np.array([700,1000,1300,1600])
        cdf = cdf.sort_values(by=['x','y'])#counts down by column
        
    idx = np.array(cdf.index)
    labels = cell_labels
    labels['circle_index']=idx
    return idx, cdf,labels
    

In [5]:
newdirs=['Kelly P15_P16_Time4_Depth2','Kelly P23_P24_Time4_Depth2','Kelly P23_P24_Time5_Depth2','Kelly P3_P8_Time3_Depth2','Kelly P3_P8_Time4_Depth2','Kelly P3_P8_Time5_Depth2','Kelly P9_P11_Time3_Depth2','Kelly P9_P11_Time4_Depth2','Kelly P9_P11_Time5_Depth2']
for version in ['V2']:#,'V1']:
    p_raw = os.path.join('/project/nsaru/raw/PySlake',version)
    p_work = os.path.join('/project/nsaru/work/PySlake',version)
    ncells=36
    D=35#mm
    if version=='V2': 
        ncells=20
        D=45
    A_mm2=D**2*pi/4

    all_samples = []
    all_matches = []
    all_areas = []
    all_circles = []
    all_labels = []
    all_times = []
    all_d = []
    for d in os.listdir(p_raw):
        if d in newdirs:
            all_samples.append(d)
            t=[]
            labels_t = []
            matches_t = []
            areas_t = []
            circles_t = []
            D_t = []
            for f in os.listdir(os.path.join(p_raw,d)):
                if '.csv' in f:
                    if f!='ImageInfo.csv':
                        cell_labels = pd.read_csv(os.path.join(p_raw,d,f))
            j = 0
            p_img = os.path.join(p_raw,d,'Images')
            if not os.path.isdir(p_img): 
                p_img = os.path.join(p_raw,d,'images')

            for idx,f in enumerate(np.sort(os.listdir(p_img))):     
                param2 = 20
                param1 = 400
                minspace = 230
                ksize = 9
                if version=='V2': 
                    minspace=270
                    ksize=15

                method = cv2.HOUGH_GRADIENT
                logging.info(d+'/'+f)
                img = cv2.imread(os.path.join(p_img,f))
                b,g,r = cv2.split(img)       
                b = cv2.GaussianBlur(b,(ksize,ksize),1)   
                retb,bt = cv2.threshold(b,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                #bt = cv2.adaptiveThreshold(b,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),0)
                h,s,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
                #retv,thresh = cv2.threshold(v,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                if d=='Kelly P24_Time1_Depth2':
                    thresh = cv2.adaptiveThreshold(v,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),65)     
                elif d=='Kelly P21_P22_P23_Time1_Depth2':
                    thresh = cv2.adaptiveThreshold(v,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),35)     
                else:
                    thresh = cv2.adaptiveThreshold(v,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),35)     
                # plt.imshow(thresh)
                # plt.show()
                # bt = bt-thresh
                # plt.imshow(bt)
                # plt.show()
                if version=='V1':
                    k = 2
                    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * k + 1, 2 * k + 1),(-1, -1)) 
                    bt = cv2.erode(bt, element)
                bt_cnt, hierarchy = cv2.findContours(image=bt, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                cnt = max(bt_cnt, key=cv2.contourArea)
                x,y,w,h = cv2.boundingRect(cnt)
                x = x-20
                y = y-20
                w = w+40
                h = h+40
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 15)
                # plt.imshow(img)
                # plt.show()
                bt = bt[y:y+h, x:x+w]
                img = img[y:y+h, x:x+w,:]
                try:
                    _,_,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
                    if d=='Kelly P24_Time1_Depth2':
                        thresh = thresh[y:y+h, x:x+w]
                        # plt.imshow(thresh)
                        # plt.show()
                    else:
                        retv,thresh = cv2.threshold(v,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                        #thresh = cv2.adaptiveThreshold(v,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),25)     
                        # plt.imshow(thresh)
                        # plt.show()

                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))
                    try:
                        #adaptive parameter selection
                        outer_max  = 5
                        ot = 0
                        while circles.shape[1]!=ncells and ot<outer_max:
                            itermax = 40        
                            it = 0
                            while circles.shape[1]!=ncells and it<itermax:
                                if circles.shape[1]<ncells:
                                    param1=param1-1
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                if circles.shape[1]>ncells:
                                    param1=param1+1
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                it=it+1
                            itermax = param2-2        
                            it = 0
                            while circles.shape[1]!=ncells and it<itermax:
                                if circles.shape[1]<ncells:
                                    param2=param2-1
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                if circles.shape[1]>ncells:
                                    param2=param2+1
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                it=it+1                
                            itermax = 10        
                            it = 0
                            while circles.shape[1]!=ncells and it<itermax:
                                if circles.shape[1]<ncells:
                                    minspace=minspace-2
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                if circles.shape[1]>ncells:
                                    minspace=minspace+2
                                    circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                                it=it+1
                            ot=ot+1

                        circles = np.uint16(np.around(circles))
                        for i in circles[0,:]:
                            # draw the outer circle
                            cv2.circle(img,(i[0],i[1]),i[2],(0,0,255),5)  

                        D_pix = 2*np.median(circles[0,:,2])
                        A_pix = D_pix**2*pi/4
                        A_convert = A_mm2/A_pix
                        idx,cdf,labels = cell_ids(circles,cell_labels)

                        contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                        # draw contours on the original image
                        cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=5, lineType=cv2.LINE_AA)

                        centers = []
                        A = []
                        for c in contours:
                            # compute the center of the contour
                            M = cv2.moments(c)
                            if M["m00"]>0:
                                cX = int(M["m10"] / M["m00"])
                                cY = int(M["m01"] / M["m00"])
                                centers.append(np.array([cX,cY]))
                                A.append(M["m00"]*A_convert)

                        cells_idx = nearest_cell(circles,centers)
                        labels = labels.merge(cells_idx, how="outer", on="circle_index")

                        #maps areas to cells
                        A_agg = np.zeros(len(labels["agg_index"].values))*np.nan
                        for idx, agid in enumerate(labels["agg_index"].values):
                            if not np.isnan(agid): A_agg[idx] = A[int(agid)]

                        labels["area"] = A_agg

                        if not os.path.isdir(os.path.join(p_work,d)): os.mkdir(os.path.join(p_work,d))
                        fout = os.path.join(p_work,d,f.split('.')[0]+'_labeled.jpg')
                        cv2.imwrite(fout,img)

                        # plt.imshow(img)
                        # plt.show()

                        labels=labels.groupby(['Column','Row','Well','Sample','Rep']).aggregate(func=sum).drop(columns=['circle_index','agg_index'])
                        matches_t.append(cells_idx)
                        areas_t.append(A)
                        circles_t.append(circles)
                        labels_t.append(labels)            
                        D_t.append(D_pix)
                        t.append(pd.to_datetime("".join(f[4:21].split('_'))))
                        j = j+1

                    except Exception as e:
                        print(e)
                        # %tb
                        # plt.subplot(121)
                        # plt.imshow(bt)
                        # plt.subplot(122)
                        # plt.imshow(img)
                        # plt.show()
                except Exception as e:
                    print(e)
                    # %tb
                    # plt.imshow(bt)
                    # plt.show()
            all_circles.append(circles_t)
            all_matches.append(matches_t)
            all_areas.append(areas_t)
            all_labels.append(labels_t)
            all_times.append(t)
            all_d.append(D_t)

    all_df = []
    for d,labels_t,t,d_t in zip(all_samples,all_labels,all_times,all_d):
        print(d)
        labels_t=[labels_t[i] for i in np.argsort(t)]
        t = np.sort(t)
        t = t-np.array(min(t))
        t = [ti.seconds for ti in t]
        df = labels_t[0]
        df = df.rename(columns={"area": str(t[0])})
        for i in range(1,len(labels_t)):
            df=df.merge(labels_t[i],how='outer',on=['Column','Row','Well','Sample','Rep'])
            df = df.rename(columns={"area": str(t[i])})
        print(os.path.join(p_work,d,d+'_'+'processed_aggregates.csv'))
        df.to_csv(os.path.join(p_work,d,d+'_'+'processed_aggregates.csv'))
        all_df.append(df)

INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_43_23_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_06_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_11_Pro.jpg


'NoneType' object has no attribute 'shape'


INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_16_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_21_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_27_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_32_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_37_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_42_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_47_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_52_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_44_58_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_03_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_08_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_13_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_18_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_24_Pro.jpg
INFO:root:Kelly P9_P11_Time3_Depth2/WIN_20230316_12_45_29_Pro.jpg
INFO:root:

'NoneType' object has no attribute 'shape'


INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_23_48_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_23_53_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_23_58_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_03_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_08_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_14_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_19_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_24_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_29_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_34_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_40_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_45_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_50_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_24_55_Pro.jpg
INFO:root:Kelly P3_P8_Time3_Depth2/WIN_20230316_12_25_00_Pro.jpg
INFO:root:Kelly P3_P8_Tim

...

'NoneType' object has no attribute 'shape'


INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_03_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_08_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_13_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_18_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_23_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_29_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_34_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_39_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_44_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_49_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_11_54_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_12_00_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_12_05_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_12_10_Pro.jpg
INFO:root:Kelly P9_P11_Time4_Depth2/WIN_20230322_16_12_15_Pro.jpg
INFO:root:

'NoneType' object has no attribute 'shape'


INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_26_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_32_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_37_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_42_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_47_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_52_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_14_58_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_03_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_08_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_13_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_18_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_23_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_29_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_34_Pro.jpg
INFO:root:Kelly P23_P24_Time5_Depth2/WIN_20230324_16_15_39_Pro

Kelly P9_P11_Time3_Depth2
/project/nsaru/work/PySlake/V2/Kelly P9_P11_Time3_Depth2/Kelly P9_P11_Time3_Depth2_processed_aggregates.csv
Kelly P3_P8_Time3_Depth2
/project/nsaru/work/PySlake/V2/Kelly P3_P8_Time3_Depth2/Kelly P3_P8_Time3_Depth2_processed_aggregates.csv
Kelly P9_P11_Time5_Depth2
/project/nsaru/work/PySlake/V2/Kelly P9_P11_Time5_Depth2/Kelly P9_P11_Time5_Depth2_processed_aggregates.csv
Kelly P15_P16_Time4_Depth2
/project/nsaru/work/PySlake/V2/Kelly P15_P16_Time4_Depth2/Kelly P15_P16_Time4_Depth2_processed_aggregates.csv
Kelly P3_P8_Time5_Depth2
/project/nsaru/work/PySlake/V2/Kelly P3_P8_Time5_Depth2/Kelly P3_P8_Time5_Depth2_processed_aggregates.csv
Kelly P23_P24_Time4_Depth2
/project/nsaru/work/PySlake/V2/Kelly P23_P24_Time4_Depth2/Kelly P23_P24_Time4_Depth2_processed_aggregates.csv
Kelly P3_P8_Time4_Depth2
/project/nsaru/work/PySlake/V2/Kelly P3_P8_Time4_Depth2/Kelly P3_P8_Time4_Depth2_processed_aggregates.csv
Kelly P9_P11_Time4_Depth2
/project/nsaru/work/PySlake/V2/Kelly P9